<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Monitoring the Status of Jobs and Backends

In this tutorial, we will learn how to monitor the status of jobs submitted to devices and simulators (collectively called backends), as well as discover how to easily query backend details and view the collective state of all the backends available to you.

### Loading the Monitoring Tools

First, let us load the default qiskit routines, and register our IBMQ credentials.

In [ ]:
from qiskit import *
IBMQ.load_accounts()

Functions for monitoring jobs and backends are here:

In [ ]:
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

If you are running inside a Jupyter notebook, then you will also need to have `ipywidgets` version `7.3.0` or higher installed.  These come pre-installed in Anaconda.  There are also Jupyter notebook 'magics' available for each of the abover functions.  The following will register those magics, making them ready for use.

In [ ]:
from qiskit.tools.jupyter import *

## Tracking Job Status

Many times a job(s) submitted to the IBM Q network can take a long time to process, e.g. jobs with many circuits and/or shots, or may have to wait in queue for other users.  In situations such as these, it is beneficial to have a way of monitoring the progress of a job, or several jobs at once.  As of Qiskit `0.6+` it is possible to monitor the status of a job in a Jupyter notebook, and also in a Python script (verision `0.7+`).

Lets see how to make use of these tools.

### Monitoring the status of a single job

Lets build a simple Bell circuit, submit it to a device, and then monitor its status.

In [ ]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.measure(q, c);

Lets grab the least busy backend

In [ ]:
from qiskit.backends.ibmq import least_busy
backend = least_busy(IBMQ.backends(simulator=False))
backend.name()

Monitor the job using `job_monitor` in blocking-mode (i.e. using the same thread as the Python interpretor)

In [ ]:
job1 = execute(qc, backend)
job_monitor(job1)

Monitor the job using `job_monitor` in async-mode (Jupyter notebooks only).  The job will be monitored in a separate thread, allowing you to continue to work in the notebook.

In [ ]:
job2 = execute(qc, backend)
job_monitor(job2, monitor_async=True)

It is also possible to monitor the job using the `qiskit_job_status` Jupyter notebook magic.  This method is always asyncronous.

In [ ]:
%%qiskit_job_status
job3 = execute(qc, backend)

Note that, for the `qiskit_job_status` to work, the job returned by `execute` must be stored in a variable so that it may be retrieved by the magic.

### Monitoring many jobs simultaneously

Here we will monitor many jobs sent the the device.  It is if the jobs are stored in a list to make retrevial easier.

In [ ]:
num_jobs = 5
my_jobs = []
for j in range(num_jobs):
    my_jobs.append(execute(qc, backend))
    job_monitor(my_jobs[j], monitor_async=True)

Or, using magic:

In [ ]:
%%qiskit_job_status
my_jobs2 = []
for j in range(num_jobs):
    my_jobs2.append(execute(qc, backend))

In the magics example, the magic is smart enough to know that the list `my_jobs2` contains jobs, and will automatically extract them and check their status.  We are not limited to using `jobs.append()`, and can use an indexed list or NumPy array as well:

In [ ]:
%%qiskit_job_status
import numpy as np
my_jobs3 = np.empty(num_jobs, dtype=object)
for j in range(num_jobs):
    my_jobs3[j] = execute(qc, backend)

### Changing the interval of status updating

By default, the interval at which the job status is checked is every two seconds.  However, the user is free to change this using the `interval` keyword argument in `job_monitor`

In [ ]:
job3 = execute(qc, backend)
job_monitor(job3, interval=5)

and the `-i` or `--interval` arguments to the Jupyter magic.

In [ ]:
%%qiskit_job_status -i 5
job4 = execute(qc, backend)

In [ ]:
%%qiskit_job_status --interval 5
job5 = execute(qc, backend)

# Backend Details

So far we have been executing our jobs on a backend, but we have explored the backends in any detail.  For example, we have found the least busy backend, but do not know if this is the best backend with respect to gate errors, topology etc.  It is possible to get detailed information for a single backend by calling `backend_monitor`:

In [ ]:
backend_monitor(backend)

Or, if we are interested in a higher-level view of all the backends available to us, then we can use `backend_overview()`

In [ ]:
backend_overview()

There are also Jupyter magic equivalents that give more detailed information.

In [ ]:
%qiskit_backend_monitor backend

The Jupyter `backend_overview` runs live in the notebook, and will automatically update itself every minute.

In [ ]:
%qiskit_backend_overview